In [ ]:
%matplotlib inline
from ccount import *
import matplotlib
import matplotlib.pyplot as plt

## Parameters

In [ ]:
ls *npy*

In [ ]:
ls *labeled*

In [ ]:
# Please change
in_name = 'xxx.npy.gz'

# Please don't change unless have to
block_height = 2048 
block_width = 2048 # pixcels

blob_extention_ratio = 1.4 # extend blob radius manually (1.4)
blob_extention_radius = 30 # pixcels to extend (2)

# Load Pre-build crops (cropped images of blobs)

In [ ]:
# load
image_flat_crops = load_blobs_db(in_name)
w = int(sqrt(image_flat_crops.shape[1]-6)) # padding width & cropped img width/2

### Histogram of blob size

In [ ]:
r_ = image_flat_crops[:,2]
 
f, (ax1, ax2) = plt.subplots(1,2, figsize=(14, 7), sharey=False)
ax1.hist(r_, 40)
ax1.semilogy()
ax1.set_title("Histogram of blob size (log10 scale)")
ax2.hist(r_, 40)
ax2.set_title("Histogram of blob size")

In [ ]:
show_rand_crops(crops=image_flat_crops, label_filter='na', num_shown=5)

## Labeling

In [ ]:
image_flat_crops = pop_label_flat_crops(image_flat_crops, random = True, seed = 1, skip_labeled = True)

## Save

In [ ]:
# save
outname_labeled = input ("please input file name to save labeled data(e.g. {})\nto exit without saving 'e':".format("xxx.labeled.npy"))
if outname_labeled == 'e':
    print('exit without saving')
else:
    np.save(outname_labeled.strip(), image_flat_crops)  
    print('saved into', outname_labeled.strip())

In [ ]:
%%bash -s "$outname_labeled"
gzip -f -1 $1

# Review Labeled DB

In [ ]:
labeled = load_blobs_db(outname_labeled.strip()+'.gz')
#labeled = image_flat_crops

In [ ]:
## labeled as YES
show_rand_crops(crops=labeled, label_filter=1, num_shown=5)

In [ ]:
## labeled as NO
show_rand_crops(crops=labeled, label_filter=0, num_shown=5)

In [ ]:
## labeled as undistinguishable
show_rand_crops(crops=labeled, label_filter=-2, num_shown=5)

In [ ]:
## any label
show_rand_crops(crops=labeled, label_filter='na', num_shown=2)

## Unsupervised classification (overview of blobs)

In [ ]:
crops = image_flat_crops
#crops = crops[crops[:, 3] >= 0, :]  # remove unlabeled
blobs_stat(crops)

In [ ]:
cluster_info = pd.DataFrame(crops[:, 3], columns=['cluster_id'])
tsne_df = pca_tsne(pd.DataFrame(crops), cluster_info=cluster_info)

In [ ]:
## save this file in html format
!jupyter nbconvert --to html labeling.ipynb